In [1]:
# TODO change difficulty score to middle being the best and 1 and 4 being the worst (too easy and too hard is bad)
# TODO filter negative comments for the worst areas and positive comments for the best areas
# TODO change the reason of a problem if section score is too high (higher than course score)

# Top problems

## Importing libraries

In [2]:
from collections import Counter
import numpy as np
import pandas as pd
import re

## Loading raw files

In [3]:
answers_raw             = pd.read_excel('./data/odpovedi_202105081142.xlsx', sheet_name='Sheet0')
courses_raw             = pd.read_excel('./data/predmety_202105081148.xlsx', sheet_name='Sheet0')
english_lemma_cloud_raw = pd.read_csv('./data/EnglishLemmaCloud.csv', usecols=['ID','lemma','SENTIMENT'])
questionnaires_raw      = pd.read_excel('./data/dotazniky_202105081150.xlsx', sheet_name='Sheet0')
questions_raw           = pd.read_excel('./data/otazky_202105081147.xlsx', sheet_name='Sheet0')
sections_raw            = pd.read_excel('./data/dot_sekce_202105081151.xlsx', sheet_name='Sheet0')
workplaces_raw          = pd.read_excel('./data/ho_pracoviste_202105081150.xlsx', sheet_name='Sheet0')

## Creating working tables from raw files

In [4]:
answers             = answers_raw.copy()
courses             = courses_raw.copy()
english_lemma_cloud = english_lemma_cloud_raw
questionnaires      = questionnaires_raw.drop(columns=['VEDLEJSKA']).drop_duplicates() # dropping VEDLEJSKA, as it duplicates ID
questions           = questions_raw.copy()
sections            = sections_raw.copy()
workplaces          = workplaces_raw.copy()

## Merging working tables into one df
## +
## Dropping unnecessary columns and renaming filtered columns

In [5]:
df = pd.merge(questionnaires, answers,             how='inner', left_on='ID',            right_on='DOTAZNIK', suffixes=['_questionnaire','_answer'])
df = pd.merge(df,             workplaces,          how='left',  left_on='HO_PRACOVISTE', right_on='ID',       suffixes=['','_workplace'])
df = pd.merge(df,             courses,             how='left',  left_on='PREDMET',       right_on='ID',       suffixes=['','_course'])
df = pd.merge(df,             english_lemma_cloud, how='left',  left_on='ID_answer',     right_on='ID',       suffixes=['','_lemma'])
df = pd.merge(df,             questions,           how='left',  left_on='OTAZKA',        right_on='ID',       suffixes=['','_question'])
df = pd.merge(df,             sections,            how='left',  left_on='DOT_SEKCE',     right_on='ID',       suffixes=['','_section'])

print(df.columns)

df.drop(inplace=True,columns=['PREDMET','HO_PRACOVISTE','PROGRAM','PROGRAM_PRACOVISTE','FORMA','OBOR','UKONCENI','VYSLEDEK','DATUM_VYPLNENI','DOBA_VYPLNENI','ID_answer','DOTAZNIK','OTAZKA','PEDAGOG','TYP_AKCE','HODNOTA_CLOB','ID','HODN_OBDOBI','AKTIVNI','PRETIZENO','NAZEV','ZKRATKA','ID_course','KOD','GARANTUJICI_PRACOVISTE','ID_lemma','ID_question','DOT_SEKCE','OTAZKA_ANG','OTAZKA_TYP','PORADI','PER_TYP_AKCE','HO_PRACOVISTE_question','POVINNA_ODPOVED','ID_section','DOT_CAST','NAZEV_ANG','PORADI_section','PER_TYP_AKCE_section','PER_PEDAGOG_section'])
df.rename(inplace=True,columns={'ID_questionnaire':'id_questionnaire', 'ID_answer':'id_answer', 'HODNOTA_CISLO':'score', 'NAZEV_course':'course', 'OBDOBI':'period', 'SENTIMENT':'sentiment' ,'OTAZKA_question':'question' ,'PER_PEDAGOG':'per_teacher' ,'NAZEV_section':'section'})

print(df.columns)

Index(['ID_questionnaire', 'PREDMET', 'HO_PRACOVISTE', 'PROGRAM',
       'PROGRAM_PRACOVISTE', 'FORMA', 'OBOR', 'UKONCENI', 'VYSLEDEK',
       'DATUM_VYPLNENI', 'DOBA_VYPLNENI', 'ID_answer', 'DOTAZNIK', 'OTAZKA',
       'PEDAGOG', 'TYP_AKCE', 'HODNOTA_CISLO', 'HODNOTA_CLOB', 'ID',
       'HODN_OBDOBI', 'AKTIVNI', 'PRETIZENO', 'NAZEV', 'ZKRATKA', 'ID_course',
       'KOD', 'NAZEV_course', 'GARANTUJICI_PRACOVISTE', 'OBDOBI', 'ID_lemma',
       'lemma', 'SENTIMENT', 'ID_question', 'DOT_SEKCE', 'OTAZKA_question',
       'OTAZKA_ANG', 'OTAZKA_TYP', 'PORADI', 'PER_PEDAGOG', 'PER_TYP_AKCE',
       'HO_PRACOVISTE_question', 'POVINNA_ODPOVED', 'ID_section', 'DOT_CAST',
       'NAZEV_section', 'NAZEV_ANG', 'PORADI_section', 'PER_TYP_AKCE_section',
       'PER_PEDAGOG_section'],
      dtype='object')
Index(['id_questionnaire', 'score', 'course', 'period', 'lemma', 'sentiment',
       'question', 'per_teacher', 'section'],
      dtype='object')


## Adding year_semester, so data can be sorted from newest to oldest

In [6]:
df[['semester','year']] = df['period'].str.split(' ',expand=True)
df['id_semester'] = np.where(df['semester']=='ZS','0','1')
df['year_semester'] = df['year'] + '_' + df['id_semester']
df.drop(inplace=True,columns=['year','id_semester'])

df.head()

,id_questionnaire,score,course,period,lemma,sentiment,question,per_teacher,section,semester,year_semester
0,357017,NaN,Informatika (v angličtině),ZS 2019/2020,NaN,NaN,Co a jak by se na výuce mohlo zlepšit (cvičení...,0,Celková spokojenost s předmětem a náměty na zl...,ZS,2019/2020_0
1,357017,NaN,Informatika (v angličtině),ZS 2019/2020,NaN,NaN,Látka probíraná v předmětu se NEPŘEKRÝVALA s j...,0,Překryv s jinými předměty,ZS,2019/2020_0
2,357017,3.0,Informatika (v angličtině),ZS 2019/2020,NaN,NaN,Celkově jsem s kvalitou předmětu spokojen/a (h...,0,Celková spokojenost s předmětem a náměty na zl...,ZS,2019/2020_0
3,357017,4.0,Informatika (v angličtině),ZS 2019/2020,NaN,NaN,Srozumitelnost výkladu a schopnost vysvětlit l...,1,Jak na mě působil vyučující z následujících hl...,ZS,2019/2020_0
4,357017,4.0,Informatika (v angličtině),ZS 2019/2020,NaN,NaN,Připravenost přednášek/cvičení,1,Jak na mě působil vyučující z následujících hl...,ZS,2019/2020_0


## Checking shapes after merging

In [7]:
print('answers_raw\t\t'           + str(answers_raw.shape))
print('english_lemma_cloud_raw\t' + str(english_lemma_cloud_raw.shape))
print('courses_raw\t\t'           + str(courses_raw.shape))
print('questionnaires_raw\t'      + str(questionnaires_raw.shape))
print('questions_raw\t\t'         + str(questions_raw.shape))
print('sections_raw\t\t'          + str(sections_raw.shape))
print('workplaces_raw\t\t'        + str(workplaces_raw.shape))
print('df\t\t\t'                  + str(df.shape))

len(answers_raw) == len(df)

answers_raw		(105410, 7)
english_lemma_cloud_raw	(6666, 3)
courses_raw		(6678, 5)
questionnaires_raw	(12706, 12)
questions_raw		(52, 10)
sections_raw		(14, 7)
workplaces_raw		(36, 6)
df			(105410, 11)


True

## Aggregating mean and count of score by course and year_semester and sorting each course from newest to oldest

In [8]:
df_grouped = df.groupby(['course', 'year_semester'])['score'].agg([('score_count', 'count'), ('score_mean', 'mean')]).reset_index().sort_values(by=['course', 'year_semester'], ascending=(True,False))
df_grouped['score_mean'] = df_grouped['score_mean'].round(2)

df_grouped.head(10)

,course,year_semester,score_count,score_mean
0,Academic Writing - odborný písemný projev (C1),2019/2020_0,9,3.78
1,Agilní projekt vývoje webové aplikace,2020/2021_0,190,3.69
2,Agilní vývoj webových aplikací,2019/2020_0,223,3.48
5,Analýza a návrh informačních systémů,2020/2021_0,863,3.53
4,Analýza a návrh informačních systémů,2019/2020_1,615,3.44
3,Analýza a návrh informačních systémů,2019/2020_0,747,3.55
8,Analýza a návrh testů softwaru,2020/2021_0,207,3.61
7,Analýza a návrh testů softwaru,2019/2020_1,266,3.47
6,Analýza a návrh testů softwaru,2019/2020_0,332,3.51
10,Analýza kategoriálních dat,2020/2021_0,58,3.55


## Creating variable for the newest semester in data

In [9]:
newest_semester = df_grouped['year_semester'].max()
newest_semester

'2020/2021_0'

## Courses, which occur only in the newest semester, are evaluated by their average score from worst to best

In [10]:
df_new_courses = df_grouped.drop_duplicates(keep=False, subset=['course'], inplace=False)
df_new_courses = df_new_courses[(df_new_courses['year_semester'] == newest_semester)].sort_values(by='score_mean')
df_new_courses['course_type'] = 'new'

df_new_courses = df_new_courses[df_new_courses['score_mean'] < 3]

df_new_courses.head(15)

,course,year_semester,score_count,score_mean,course_type
271,Posilování,2020/2021_0,8,1.00,new
437,Šerm,2020/2021_0,4,1.00,new
365,"Uznání zápočtu TV z jiné fakulty, VŠ 1",2020/2021_0,4,1.00,new
407,Úvod do programování v jazyce Python,2020/2021_0,44,2.32,new
320,Softwarové architektury,2020/2021_0,212,2.58,new
252,Pilates,2020/2021_0,4,2.75,new
327,Sportovní lezení,2020/2021_0,32,2.75,new
230,Neplavci a slabí plavci,2020/2021_0,5,2.80,new
410,Účetnictví I.,2020/2021_0,35,2.94,new


## Courses, which occur in the newest semester as well as in at least one other, are evaluated by the absolute diff between newest semester and the previous one (whenever it was) absolute diff is used, so drop from 2 to 1 has the same weight as drop from 4 to 3

In [11]:
df_existing_courses = df_grouped[df_grouped['course'].duplicated(keep=False)].copy()
df_existing_courses['rank'] = df_existing_courses.groupby(['course'])['year_semester'].cumcount()
df_existing_courses = df_existing_courses[df_existing_courses['rank'] < 2].drop(columns=['rank']) # keeping only last two semesters for each course
df_existing_courses['score_mean_previous'] = df_existing_courses.groupby('course')['score_mean'].shift(-1) # adding score from the previous semester to the following one
df_existing_courses = df_existing_courses[df_existing_courses['year_semester'] == newest_semester] # keeping only the newest (current) semester
df_existing_courses['score_mean_diff'] = (df_existing_courses['score_mean'] - df_existing_courses['score_mean_previous'])
df_existing_courses['course_type'] = 'existing'
df_existing_courses = df_existing_courses.sort_values(by='score_mean_diff')

df_existing_courses = df_existing_courses[df_existing_courses['score_mean_diff'] < -0.5]

df_existing_courses.head(15)

,course,year_semester,score_count,score_mean,score_mean_previous,score_mean_diff,course_type
131,Hokej,2020/2021_0,4,1.75,3.67,-1.92,existing
353,Thai box,2020/2021_0,8,2.12,3.88,-1.76,existing
297,Právo počítačové a informačních a komunikačníc...,2020/2021_0,44,2.75,3.62,-0.87,existing
172,Jak uspět v médiích,2020/2021_0,10,2.70,3.56,-0.86,existing
124,Funkční kruhový trénink,2020/2021_0,42,3.14,3.96,-0.82,existing
163,International Week - KIZI,2020/2021_0,10,2.30,2.88,-0.58,existing
371,Užití MS Excelu v podnikové praxi (v angličtině),2020/2021_0,129,3.28,3.85,-0.57,existing
425,Řízení podnikové výkonnosti v nástrojích CPM a...,2020/2021_0,128,3.06,3.62,-0.56,existing
379,Využití ICT ve finančním účetnictví,2020/2021_0,28,2.57,3.11,-0.54,existing


## Appending new and existing courses

In [12]:
df_new_and_existing_courses = pd.concat([df_new_courses, df_existing_courses],sort=False)

len(df_new_and_existing_courses) == len(df_new_courses) + len(df_existing_courses)

True

## Adding reasons, why the worst courses are the worst

In [13]:
for index, row in df_new_and_existing_courses.iterrows():
    
    # sections and their scores
    
    df_worst_sections = df[(df['course'] == row['course']) & (df['year_semester'] == newest_semester)].groupby(['section'])['score'].agg([('score_mean','mean')]).reset_index()
    df_worst_sections = df_worst_sections[df_worst_sections['score_mean'].notna()].sort_values(by='score_mean')
    worst_section = df_worst_sections['section'][0]
    worst_section_score = df_worst_sections['score_mean'][0].round(2)
    
    # most frequent comments
    
    lemma = df[(df['course'] == row['course']) & (df['year_semester'] == newest_semester) & (df['lemma'].notna()) & (df['section'] == worst_section)]['lemma'].copy()
    lemma = lemma.str.split()
    lemma_frequency = Counter([item for sublist in lemma.tolist() for item in sublist]).most_common()
    lemma_frequency_string = str([x for x in lemma_frequency])
    words_with_frequency_list = re.sub("[^\w]", " ",  lemma_frequency_string).split()
    most_frequent_words_list = [x for x in words_with_frequency_list if not x.isdigit()][:10]
    most_frequent_words = ', '.join(map(str, most_frequent_words_list))
    
    # creating columns with the worst section with corresponding score and the most frequent comments
    
    df_new_and_existing_courses.loc[index,'reason_name'] = worst_section
    df_new_and_existing_courses.loc[index,'reason_value'] = worst_section_score
    df_new_and_existing_courses.loc[index,'reason_text'] = most_frequent_words

## Testing results

In [14]:
for index, row in df_new_and_existing_courses.iterrows():
    if row['course_type'] == 'new':
        print(row['course'] + ' has a bad score (' + str(row['score_mean']) + '), because of ' + row['reason_name'] + ' (score: ' + str(row['reason_value']) + ').\nMost frequent words in this section: ' + row['reason_text'])
        print()
    elif row['course_type'] == 'existing':
        print(row['course'] + ' score got worse by ' + str(row['score_mean_diff']) + ', because of ' + row['reason_name'] + ' (score: ' + str(row['reason_value']) + ').\nMost frequent words in this section: ' + row['reason_text'])
        print()

Posilování has a bad score (1.0), because of Celková spokojenost s předmětem a náměty na zlepšení (score: 1.0).
Most frequent words in this section: quarantine, improve, taught, D, competence, school, teacher, home, practice, dinghy

Šerm has a bad score (1.0), because of Celková spokojenost s předmětem a náměty na zlepšení (score: 1.0).
Most frequent words in this section: 

Uznání zápočtu TV z jiné fakulty, VŠ 1 has a bad score (1.0), because of Celková spokojenost s předmětem a náměty na zlepšení (score: 1.0).
Most frequent words in this section: 

Úvod do programování v jazyce Python has a bad score (2.32), because of Celková spokojenost s předmětem a náměty na zlepšení (score: 1.22).
Most frequent words in this section: focus, data, program, teach, object, content, foundation, Python, work, create

Softwarové architektury has a bad score (2.58), because of Celková spokojenost s předmětem a náměty na zlepšení (score: 1.82).
Most frequent words in this section: clear, time, semester